<a href="https://colab.research.google.com/github/KevinHern/AI-Crash-Course/blob/main/AI_Crash_Course_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks

[Presentation: AI Crash Course 06](https://view.genial.ly/619fcd2a0ae1200db0d2d01c/presentation-ai-crashcourse06)

## 0) Preparations

In [1]:
# ----- Libraries ----- #

# For graph plotting
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

# For dataset manipulation
import numpy as np

# For visualizing more complex graphs
import seaborn as sns

# Neural networks
import tensorflow as tf

# Global constant for training acceleration
AUTOTUNE = tf.data.AUTOTUNE

## 1) Dataset Preparations

In [ ]:
# Lets use an already available dataset
from tensorflow.keras import datasets

# Lets try to classify numbers between 0 and 9
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# A good practice when dealing with images: Normalize images from 0 to 1 or from -1 to 1.
# We do not want our neural net to have big weights → one slight change and the outcome may be completely different!
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# Lets set the classes that the dataset contains
class_names = ['0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9']

# Lets visualize a few samples
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i]])
plt.show()

# Model

## 2) Callbacks

In [ ]:
'''
  CALLBACKS:
  These are functions that are called once a certain amount of epochs end. These come very handful to deal with overfitting
'''

# Lets load an extension that helps us to visualize the performance of our model
%load_ext tensorboard

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/model/', histogram_freq=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('logs/model/', monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3,)

## 3) Construction

In [ ]:
# Lets build the model. NOTE: this is the construction of the architecture of the model!
model = tf.keras.models.Sequential([
  # Feature Extraction
  tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1) ,activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1) ,activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)),
  tf.keras.layers.BatchNormalization(),

  # NN for classification
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=128, activation='relu'),
  tf.keras.layers.Dropout(rate=0.15),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=len(class_names), activation='sigmoid')
  ])

# Now lets compile the model. NOTE: These are the finishing touches before having a fully functional model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Now lets train the model!
model.fit(train_images,
          train_labels,
          epochs=100,
          batch_size = 128,
          validation_split=0.2,
          callbacks=[tensorboard_callback, checkpoint_callback, earlystopping_callback]
        )

# Lets evaluate our model
model.evaluate(x=test_images, y=test_labels, batch_size=128)

In [ ]:
# Lets visualize
%tensorboard --logdir "logs/model/" --port=8008

## 4) Confusion Matrix

In [19]:
# Function that plots confusion matrix
def plot_confusion_matrix(labels, predictions):
  figure = plt.figure(figsize=(12, 12))
  sns.heatmap(confusion_matrix(labels=labels, predictions=predictions), annot=True,cmap=plt.cm.Blues)
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

In [ ]:
predictions = model.predict(x=test_images)
predictions = list(map(lambda x: np.argmax(x), predictions))

plot_confusion_matrix(labels=test_labels, predictions=predictions)